In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import os
import time
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt

In [ ]:
def load_image(image_path):
    img = load_img(image_path, target_size=(256, 256))
    img = img_to_array(img)
    img = (img / 127.5) - 1  # Normalize image to [-1, 1]
    return img

def load_dataset(dataset_path):
    images = []
    for filename in os.listdir(dataset_path):
        img = load_image(os.path.join(dataset_path, filename))
        images.append(img)
    return np.array(images)


In [ ]:
def residual_block(x, filters, size=3, stride=1):
    
    res = layers.Conv2D(filters, size, strides=stride, padding="same")(x)
    res = layers.BatchNormalization()(res)
    res = layers.LeakyReLU(0.2)(res)

    res = layers.Conv2D(filters, size, strides=stride, padding="same")(res)
    res = layers.BatchNormalization()(res)
    return layers.add([x, res])

def build_generator(input_shape=(256, 256, 3)):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, 7, strides=1, padding="same")(inputs)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.BatchNormalization()(x)

    # Down-sampling layers
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(256, 3, strides=2, padding="same")(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.BatchNormalization()(x)

    # Residual Blocks
    for _ in range(6):
        x = residual_block(x, 256)

    # Upsampling layers
    x = layers.Conv2DTranspose(128, 3, strides=2, padding="same")(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2DTranspose(64, 3, strides=2, padding="same")(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.BatchNormalization()(x)

    outputs = layers.Conv2D(3, 7, strides=1, padding="same", activation='tanh')(x)

    model = Model(inputs, outputs)
    return model


In [ ]:
def build_discriminator():
    inputs = layers.Input(shape=[256, 256, 3])

    x = layers.Conv2D(64, 4, strides=2, padding='same')(inputs)
    x = layers.LeakyReLU(0.2)(x)
    
    x = layers.Conv2D(128, 4, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(256, 4, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(512, 4, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(1, 4, strides=1, padding='same')(x)

    model = Model(inputs, x)
    return model


In [ ]:
def cycle_loss(real_image, cycled_image):
    return tf.reduce_mean(tf.abs(real_image - cycled_image))

def discriminator_loss(disc_real_output, disc_generated_output):
    disc_real_output = tf.reshape(disc_real_output, [-1])  # Flattening the output
    disc_generated_output = tf.reshape(disc_generated_output, [-1])  # Flattening the output
    
    real_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(disc_real_output), logits=disc_real_output))
    generated_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(disc_generated_output), logits=disc_generated_output))
    total_disc_loss = (real_loss + generated_loss) * 0.5
    return total_disc_loss

def generator_loss(disc_generated_output, cycled_image, real_image, lambda_cycle=10.0):
    gan_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(disc_generated_output), logits=disc_generated_output))
    cycle_loss_value = cycle_loss(real_image, cycled_image)
    total_gen_loss = gan_loss + (lambda_cycle * cycle_loss_value)
    return total_gen_loss


In [ ]:
generator_g = build_generator()
generator_f = build_generator()
discriminator_x = build_discriminator()
discriminator_y = build_discriminator()

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.0002, decay_steps=100000, decay_rate=0.96, staircase=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.5)


In [ ]:
@tf.function
def train_step(real_x, real_y):
    with tf.GradientTape() as tape:

        real_x = tf.expand_dims(real_x, axis=0)  # Shape becomes (1, 256, 256, 3)
        real_y = tf.expand_dims(real_y, axis=0)  # Shape becomes (1, 256, 256, 3)
        
        # Generate fake images
        fake_y = generator_g(real_x)  # Generate winter image from summer
        fake_x = generator_f(real_y)  # Generate summer image from winter
        
        # Cycle consistency loss
        cycled_x = generator_f(fake_y)
        cycled_y = generator_g(fake_x)
        
        # Identity loss (optional, if included in CycleGAN)
        identity_loss = tf.reduce_mean(tf.abs(real_y - fake_y)) + tf.reduce_mean(tf.abs(real_x - fake_x))
        
        # Define the discriminator losses, generator losses, and cycle consistency losses
        disc_x_loss = discriminator_x(real_x, fake_x)
        disc_y_loss = discriminator_y(real_y, fake_y)
        
        gen_g_loss = generator_loss(disc_y_loss, cycled_y, real_x)  # Generator loss for generating fake winter images
        gen_f_loss = generator_loss(disc_x_loss, cycled_x, real_y)  # Generator loss for generating fake sumer images
        
        # Total loss (including identity loss and cycle consistency loss if used)
        total_gen_loss = gen_g_loss + gen_f_loss + identity_loss
        
    return gen_g_loss, gen_f_loss, disc_x_loss, disc_y_loss

In [ ]:
def display_images(real_x, fake_y, real_y , fake_x, num_images):
    plt.figure(figsize=(10, 10))

    for i in range(num_images):
        # Plot real summer images
        plt.subplot(num_images, 2, 2 * i + 1)
        plt.imshow((real_x[i] + 1) / 2)  # Rescale to [0, 1]
        plt.title(f"Real Summer {i+1}")
        plt.axis('off')

        # Plot fake winter images
        plt.subplot(num_images, 2, 2 * i + 2)
        plt.imshow((fake_y[i] + 1) / 2)  # Rescale to [0, 1]
        plt.title(f"Fake Winter {i+1}")
        plt.axis('off')
        
    for i in range(num_images):
        # Plot real winter images
        plt.subplot(num_images, 2, 2 * i + 1)
        plt.imshow((real_y[i] + 1) / 2)  # Rescale to [0, 1]
        plt.title(f"Real Winter {i+1}")
        plt.axis('off')

        # Plot fake summer images
        plt.subplot(num_images, 2, 2 * i + 2)
        plt.imshow((fake_x[i] + 1) / 2)  # Rescale to [0, 1]
        plt.title(f"Fake Summer {i+1}")
        plt.axis('off')
        
    plt.tight_layout()
    plt.show()

def train(dataset_x, dataset_y, num_images=10):

        for real_x, real_y in zip(dataset_x, dataset_y):
            # Train the model with real summer and winer images
            gen_g_loss, gen_f_loss, disc_x_loss, disc_y_loss = train_step(real_x, real_y)

            selected_indices = np.random.randint(len(dataset_x), size=num_images)
            selected_real_x = dataset_x[selected_indices]  # 10 randomly selected summer images

            fake_y = generator_g(selected_real_x)  # Generate fake winter images

            selected_indices = np.random.randint(len(dataset_y), size=num_images)
            selected_real_y = dataset_y[selected_indices]  # 10 randomly selected winter images

            fake_x = generator_f(selected_real_y)  # Generate fake summer images

            # Display real vs fake images
            display_images(selected_real_x, fake_y, selected_real_y, fake_x, num_images)

        

In [ ]:
# Load datasets
summer_dataset = load_dataset('/kaggle/input/summer2winter-yosemite/trainA') 
winter_dataset = load_dataset('/kaggle/input/summer2winter-yosemite/trainB')

# Train the model
train(summer_dataset, winter_dataset, num_images=10)
